# Fire Detection Project
In this project we will train YOLOv10 model to detect fire and smoke in images and videos
A question that anyone will ask himself is why we need this project. As you can notice that quite all public and private facilities have security cameras, so we can integrate this model with these cameras so we can detect fires quickly and automatically send a signal to the fire department with the degree of danger and the location of the fire either that is inside a building, in the streets, etc.
We will follow these steps to train the model: <br>
1- Download the dataset from roboflow <br>
2- Explore and analyze the dataset <br>
3- Train the model <br>
4- Test the model <br>

In [2]:
#Important imports

from ultralytics import YOLOv10
from roboflow import Roboflow
import supervision as sv
import cv2
import os
import shutil
from random import shuffle

### Download the dataset

In [3]:
rf = Roboflow(api_key="G94TrvQsmmDIdcV86fkT")
project = rf.workspace("fire-dataset-tp9jt").project("fire-detection-sejra")
version = project.version(1)
dataset = version.download("yolov8")
print(dataset.location)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.34, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Fire-Detection-1 in yolov8:: 100%|██████████| 17884/17884 [00:14<00:00, 1213.91it/s]

c:\Users\user\Downloads\yolov10-main\Fire-Detection-1


### Explore the dataset
All the images have  the size of 416*416 and I think the data doesn't need any preprocessing. <br>
The ground truth has 3 classes, fire, smoke and other. This class "other" will just confuse the model because anything but fire and smoke should be considered as background so we will remove any "other" class in the ground truth. <br>
Modify the data.yaml file

In [4]:
# Removing class "other" from the ground truth
# Changing the class "smoke" from 2 to 1
all_labels_dir = "Fire-Detection-1/train/labels"

for file in os.listdir(all_labels_dir):
    with open(os.path.join(all_labels_dir, file), "r") as f:
        lines = f.readlines()

    with open(os.path.join(all_labels_dir, file), "w") as f:
        for line in lines:
            if not line.startswith("1"):
                if line.startswith("2"):
                    line = "1" + line[1:]
                f.write(line)

In [5]:
# Make the validation and test sets
# We will take 15% of the data for validation and 15% for the test set.

all_images_dir = "Fire-Detection-1/train/images"
img_names = os.listdir(all_images_dir)
shuffle(img_names)
start_point = 0
end_point = int(0.15 * len(img_names))

for folder in ["valid", "test"]:
    # Create the validation and test folders and two folders for images and labels
    os.makedirs(f"Fire-Detection-1/{folder}/images")
    os.makedirs(f"Fire-Detection-1/{folder}/labels")

    images = img_names[start_point:end_point]

    for image in images:
        shutil.move(f"{all_images_dir}/{image}", f"Fire-Detection-1/{folder}/images/{image}")
        label = image.replace(".jpg", ".txt")
        shutil.move(f"{all_labels_dir}/{label}", f"Fire-Detection-1/{folder}/labels/{label}")

    start_point = end_point
    end_point += int(0.15 * len(img_names))

### Train the YOLOv10 model

In [2]:
model = YOLOv10("yolov10b.pt")

model.train(data=f'../Fire-Detection-1/data.yaml', epochs=100, batch=16, imgsz=416, plots=True, amp=True)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov10b.pt, data=../Fire-Detection-1/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

train: Scanning C:\Users\user\Downloads\yolov10-main\Fire-Detection-1\train\labels.cache... 6259 images, 732 backgrounds, 0 corrupt: 100%|██████████| 6259/6259 [00:00<?, ?it/s]
val: Scanning C:\Users\user\Downloads\yolov10-main\Fire-Detection-1\valid\labels.cache... 1340 images, 150 backgrounds, 0 corrupt: 100%|██████████| 1340/1340 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 135 weight(decay=0.0), 148 weight(decay=0.0005), 147 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 6 dataloader workers
Logging results to runs\detect\train2
Starting training for 100 epochs...

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


      1/100      6.04G      1.907      2.242      1.954      1.904      2.847      1.843          9        416: 100%|██████████| 392/392 [01:38<00:00,  3.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:13<00:00,  3.15it/s]


                   all       1340       2820      0.276       0.24       0.18     0.0625

      Epoch    GPU_mem     box_om     cls_om     dfl_om     box_oo     cls_oo     dfl_oo  Instances       Size


...

    100/100      6.01G      1.191     0.9143      1.395      1.289       1.02      1.407          2        416: 100%|██████████| 392/392 [01:26<00:00,  4.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:12<00:00,  3.25it/s]


                   all       1340       2820      0.562      0.453      0.478      0.235

100 epochs completed in 2.872 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 41.4MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 41.4MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
YOLOv10b summary (fused): 383 layers, 20414236 parameters, 0 gradients, 97.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 42/42 [00:12<00:00,  3.34it/s]


                   all       1340       2820      0.601      0.463      0.509      0.254
                  fire       1340       1988      0.614      0.576      0.606      0.312
                 smoke       1340        832      0.588       0.35      0.412      0.196
Speed: 0.1ms preprocess, 6.3ms inference, 0.0ms loss, 0.1ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001FE57F86A60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

### Test the model

In [4]:
model = YOLOv10("runs/detect/train2/weights/best.pt")

images_directory = "Fire-Detection-1/test/images"
labels_directory = "Fire-Detection-1/test/labels"
labels = os.listdir(labels_directory)

results = model(source=images_directory, conf=0.25)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/1340 c:\Users\user\Downloads\yolov10-main\Fire-Detection-1\test\images\large_-100-_jpg.rf.2f25938357fb7f31ddf960c5c6bb7383.jpg: 416x416 2 fires, 11.0ms
image 2/1340 c:\Users\user\Downloads\yolov10-main\Fire-Detection-1\test\images\large_-1010-_jpg.rf.4aa11e5844b15a5205421da98c62204c.jpg: 416x416 1 fire, 2 smokes, 14.0ms
image 3/1340 c:\Users\user\Downloads\yolov10-main\Fire-Detection-1\test\images\large_-1011-_jpg.rf.8fa292488f24151800d461405ea

In [38]:
def convert_from_yolo(annotations_file, image_width, image_height):
    """
    Convert annotations from YOLO format to a single bounding box.
    Args:
        annotation (list): A list of annotations in YOLO format [class_id, x_center, y_center, width, height].
    Returns:
        list: A list of annotations in the format [class_id, x_min, y_min, x_max, y_max].
    """
    with open(annotations_file, 'r') as f:
        annotations = [[float(item) for item in line.strip().split()]
                       for line in f.readlines()]

    # Convert annotations from YOLO format to a single bounding box
    for i in range(len(annotations)):
        class_id, x_center, y_center, width, height = annotations[i]
        x_center *= image_width
        y_center *= image_height
        width *= image_width
        height *= image_height
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        annotations[i] = [class_id, x_min, y_min, x_max, y_max]
    return annotations

In [37]:
# Here, we will plot the detections and the ground truth to the images to check if the models works well.

boundingbox_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()

for result in results:
    image = result.orig_img
    detections = sv.Detections.from_ultralytics(result)
    image = boundingbox_annotator.annotate(image, detections)
    image = label_annotator.annotate(image, detections)

    annotations_file = result.path.replace(".jpg", ".txt").replace("images", "labels")
    image_name = result.path.split("\\")[-1]
    ground_truth = convert_from_yolo(annotations_file, *result.orig_shape)

    for box in ground_truth:
        box_color = (0, 255, 255) if box[0] == 0 else (0, 0, 0)
        cv2.rectangle(image, tuple(box[1:3]), tuple(box[3:]), box_color, 2)
    cv2.imwrite(f"runs/detect/train2/test_results/{image_name}", image)

In [40]:
results = model(source="videoplayback.mp4", conf=0.25, save=True, imgsz=416)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/918) c:\Users\user\Downloads\yolov10-main\videoplayback.mp4: 256x416 1 fire, 1 smoke, 115.0ms
video 1/1 (frame 2/918) c:\Users\user\Downloads\yolov10-main\videoplayback.mp4: 256x416 1 fire, 1 smoke, 84.0ms
video 1/1 (frame 3/918) c:\Users\user\Downloads\yolov10-main\videoplayback.mp4: 256x416 1 fire, 1 smoke, 91.0ms
video 1/1 (frame 4/918) c:\Users\user\Downloads\yolov10-main\videoplayback.mp4: 256x416 1 fire, 1 smoke, 34.0ms
video 1